In [ ]:
from pycocotools.coco import COCO
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pycocotools.mask as mask_util
from random import randint

coco_folder = '/mnt/coco/'
dp_coco = COCO( coco_folder + 'annotations/DensePose_COCO/densepose_coco_2014_minival.json')

In [ ]:
# Get img id's for the minival dataset.
im_ids = dp_coco.getImgIds()
# Select a random image id.
Selected_im = im_ids[randint(0, len(im_ids))] # Choose im no 57 to replicate 
# Load the image
im = dp_coco.loadImgs(Selected_im)[0]  
# Load Anns for the selected image.
ann_ids = dp_coco.getAnnIds( imgIds=im['id'] )
anns = dp_coco.loadAnns(ann_ids)

In [ ]:
# Now read and b
im_name = os.path.join( coco_folder + 'images/val2017', im['file_name'][13:] )

In [ ]:
I=cv2.imread(im_name)
plt.imshow(I[:,:,::-1]); plt.axis('off'); plt.show()

In [ ]:
img_info = im

In [ ]:
def GetDensePoseMask(Polys):
    MaskGen = np.zeros([256,256])
    for i in range(1,15):
        if(Polys[i-1]):
            current_mask = mask_util.decode(Polys[i-1])
            MaskGen[current_mask>0] = i
    return MaskGen

In [ ]:
I_vis=I.copy()/2 # Dim the image.

for ann in anns:  
    bbr =  np.array(ann['bbox']).astype(int) # the box.
    if( 'dp_masks' in ann.keys()): # If we have densepose annotation for this ann, 
        Mask = GetDensePoseMask(ann['dp_masks'])
        ################
        x1,y1,x2,y2 = bbr[0],bbr[1],bbr[0]+bbr[2],bbr[1]+bbr[3]
        x2 = min( [ x2,I.shape[1] ] );  y2 = min( [ y2,I.shape[0] ] )
        ################ 
        MaskIm = cv2.resize( Mask, (int(x2-x1),int(y2-y1)) ,interpolation=cv2.INTER_NEAREST)
        MaskBool = np.tile((MaskIm==0)[:,:,np.newaxis],[1,1,3])
        #  Replace the visualized mask image with I_vis.
        Mask_vis = cv2.applyColorMap( (MaskIm*15).astype(np.uint8) , cv2.COLORMAP_PARULA)[:,:,:]
        Mask_vis[MaskBool]=I_vis[y1:y2,x1:x2,:][MaskBool]
        I_vis[y1:y2,x1:x2,:] = I_vis[y1:y2,x1:x2,:]*0.3 + Mask_vis*0.7

plt.imshow(I_vis[:,:,::-1].astype(int)); plt.axis('off'); plt.show()

In [ ]:
ann_info = anns

In [19]:
img_info

{'license': 1,
 'file_name': 'COCO_val2014_000000301563.jpg',
 'coco_url': 'http://mscoco.org/images/301563',
 'height': 428,
 'width': 640,
 'date_captured': '2013-11-20 06:07:09',
 'flickr_url': 'http://farm4.staticflickr.com/3423/3983648748_94a0b36ddd_z.jpg',
 'id': 301563}

In [34]:
anns[0].keys()

dict_keys(['segmentation', 'num_keypoints', 'dp_masks', 'area', 'dp_I', 'iscrowd', 'keypoints', 'dp_x', 'dp_U', 'image_id', 'dp_V', 'bbox', 'category_id', 'dp_y', 'id'])

In [32]:
ann_info = {}

In [27]:
ann_info['parsing'] = [ann_info[i]['dp_masks'] if 'dp_masks' in ann_info[i].keys() else [] for i in range(len(ann_info))]
ann_info['bboxes'] = [ann_info[i]['bbox'] if 'dp_masks' in ann_info[i].keys() else [] for i in range(len(ann_info))]

TypeError: list indices must be integers or slices, not str

In [14]:
results = {'img_info': img_info, 'ann_info': ann_info}

In [15]:
 def dp_mask_to_mask(polys):
        
    semantic_mask = np.zeros((256, 256), dtype=np.uint8)
    if len(polys) < 14:
        return semantic_mask
    for i in range(1, 15):
        if polys[i-1]:
            current_mask = maskUtils.decode(polys[i - 1])
            semantic_mask[current_mask > 0] = i

    return semantic_mask
    
def rle2parsing(self, mask_ann, h, w, bbox):
    # encoded dp_mask
    semantic_mask = np.zeros((h, w), dtype=np.uint8)
    mask = dp_mask_to_mask(mask_ann)
    bbr = np.array(bbox).astype(int)
    x1, y1, x2, y2 = bbr[0], bbr[1], bbr[0] + bbr[2], bbr[1] + bbr[3]
    x2, y2 = min(x2, w), min(y2, h)
    if x1 < x2 and y1 < y2:
        mask = cv2.resize(mask, (int(x2 - x1), int(y2 - y1)),
                                  interpolation=cv2.INTER_NEAREST)
        mask_bool = np.where(mask > 0, 1, 0)
        semantic_mask[y1:y2, x1:x2][mask_bool > 0] = mask[mask_bool > 0]
        mask = semantic_mask
    return mask

In [16]:
RLE2parsing = True
def load_parsing(results):
    h, w = results['img_info']['height'], results['img_info']['width']
    gt_parsing = results['ann_info']['parsing']
    if gt_parsing == None:
        return None
    if RLE2parsing:
        gt_parsing = [rle2parsing(gt_parsing[i], h, w, results['gt_bboxes'][i]) for i in range(len(gt_parsing))]
    results['gt_parsing'] = gt_parsing
    results['parsing_fields'].append('gt_parsing')
    return results

In [17]:
load_parsing(results)

TypeError: list indices must be integers or slices, not str